In [1]:
import tensorflow as tf
import numpy as np
import glob 
import os
import matplotlib.pyplot as plt
import matplotlib
import scipy

In [2]:
gpus = tf.config.experimental.list_physical_devices( device_type = 'GPU' )
# 设置该程序可见的GPU：写到最前面！
tf.config.experimental.set_visible_devices( devices = gpus[2:4], device_type = 'GPU' )

### 1. 导入所有文件的路径：

In [3]:
# 所有训练图片的地址：
image_path = glob.glob( '/home/gaoboyu/学习数据集/沙漠数据集12_450/图像/*' )
label_path = glob.glob( '/home/gaoboyu/学习数据集/沙漠数据集12_450/处理后标签/*' )
# 排序：
image_path.sort( key = lambda x:int(x.split('/')[-1].split('.')[0].split('_')[-1]) )
label_path.sort( key = lambda x:int(x.split('/')[-1].split('.')[0].split('_')[-1]) )

In [4]:
image_count = len(image_path)
len(image_path), len(label_path)

(450, 450)

In [5]:
# 最后数据的乱序：
index = np.random.permutation( image_count )
image_path = np.array(image_path)[index]
label_path = np.array(label_path)[index]

### 2. 创建数据集：

In [6]:
dataset = tf.data.Dataset.from_tensor_slices( (image_path, label_path) )

In [7]:
# 划分训练、测试集
val_count = int( image_count*0.2 )
train_count = image_count - val_count

train_dataset = dataset.skip(val_count)  # 跳过前这么多数据
val_dataset = dataset.take(val_count)   # 取前面这么多数据

val_count, train_count

(90, 360)

In [8]:
# 查看：
train_dataset, val_dataset

(<SkipDataset shapes: ((), ()), types: (tf.string, tf.string)>,
 <TakeDataset shapes: ((), ()), types: (tf.string, tf.string)>)

### 3. 预处理函数：

In [9]:
# 读取图片：3通道
def read_image(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels = 3)
    return img

In [10]:
# 读取标签：1通道（每个像素都是标签值）
def read_label(path):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels = 1)
    return img

In [11]:
# 对训练数据及标签：随机翻转 —— 训练数据专用！
def random_flip(img, label):
    img_label = tf.concat( [img, label], axis = -1 )  # 图像与标签合并
    img_label = tf.image.resize(img_label, [256,256])
    img_label = tf.image.random_flip_left_right(img_label)  # 随机左右翻转
    img_label = tf.image.random_flip_up_down(img_label)     # 随机上下翻转
    return img_label[:,:,0:3], img_label[:,:,3:]

In [12]:
# 所有图像归一化 + 标签转数据类型：
def normal(img, label):
    img = tf.cast(img, tf.float32) / 127.5 - 1
    label = tf.cast(label, tf.int32)  # 不能减1，因为里面本身就有0的标签！ 
    return img, label

In [13]:
# 训练数据加载：需要裁减！
def load_train_image(img_path, label_path):
    # 获取图片与标签：
    img = read_image(img_path)
    label = read_label(label_path)
    # 预处理：随机翻转 —— 自带统一形状！
    img, label = random_flip(img, label)
    # 返回归一化：
    return normal(img, label)

def load_val_image(img_path, label_path):
    # 获取图片与标签：
    img = read_image(img_path)
    label = read_label(label_path)
    # 统一形状：
    img = tf.image.resize(img, [256,256])
    label = tf.image.resize(label, [256,256])
    # 返回归一化：
    return normal(img, label)

In [14]:
# 将数据应用于函数：
AUTOTUNE = tf.data.experimental.AUTOTUNE   # 多线程加载：图像加载是cpu处理的，cpu可以做多线程！

train_dataset = train_dataset.map( load_train_image, num_parallel_calls = AUTOTUNE )
val_dataset = val_dataset.map( load_val_image, num_parallel_calls = AUTOTUNE )

In [15]:
train_dataset, val_dataset

(<ParallelMapDataset shapes: ((256, 256, 3), (256, 256, 1)), types: (tf.float32, tf.int32)>,
 <ParallelMapDataset shapes: ((256, 256, 3), (256, 256, 1)), types: (tf.float32, tf.int32)>)

In [16]:
# 乱序、划分数据集：
# cashe()：新操作，文件不断放到缓存中，加速！
# prefetch()：新操作，GPU内存不等待，不断预处理图片
BATCH_SIZE = 10
train_dataset = train_dataset.cache().shuffle(train_count).repeat().batch(BATCH_SIZE).prefetch(AUTOTUNE)
val_dataset = val_dataset.cache().batch(BATCH_SIZE)

In [17]:
train_dataset, val_dataset

(<PrefetchDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 1)), types: (tf.float32, tf.int32)>,
 <BatchDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 1)), types: (tf.float32, tf.int32)>)

### 4. 使用预训练网络：

In [19]:
model = tf.keras.models.load_model( 'model12_450_调过拟合.h5' )

ResourceExhaustedError: OOM when allocating tensor with shape[3,3,512,1024] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc [Op:RandomUniform]

In [ ]:
# 网络整体不可训练：
model.trainable = False
# 设定最后两个卷积层可训练：
model.get_layer('dropout_x2_1').trainable = True
model.get_layer('dropout_x2_2').trainable = True
model.get_layer('dropout_x4_1').trainable = True
model.get_layer('dropout_x4_2').trainable = True
model.get_layer('dropout_x6_1').trainable = True
model.get_layer('dropout_x6_2').trainable = True
model.get_layer('dropout_x8_1').trainable = True
model.get_layer('dropout_x8_1').trainable = True
model.get_layer('dropout_x10_1').trainable = True
model.get_layer('dropout_x10_2').trainable = True
model.get_layer('output').trainable = True

In [ ]:
# 模型编译
model.compile(
    optimizer = tf.keras.optimizers.Adam( learning_rate = 0.001 / 5 ),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

In [ ]:
# 定义保存模型的回调函数：保存整个模型、只保存最好的！
# 设置保存的路径：
checkpoint_path = '/home/gaoboyu/学习保存的模型/77'
# 设置回调函数保存模型：没设置的参数都默认
cp_callback_model = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_path,
    monitor = 'val_acc',
    save_best_only = True  # 监控的目标：如果新的epoch结果比前一个要好，那就重新保存最新的，删掉旧的！
)

In [ ]:
# 定义动态学习速率调整：连续20个epoch都不提高，学习速率降低一半！
cp_callback_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_acc',
    factor = 0.1,
    patience = 20
)

In [25]:
# 模型训练
EPOCHES = 100
history = model.fit(
    train_dataset,
    epochs = EPOCHES,
    steps_per_epoch = train_count // BATCH_SIZE,
    validation_data = val_dataset,
    validation_steps = val_count // BATCH_SIZE,
    # 回调函数：
    callbacks = [cp_callback_model, cp_callback_lr]
)

Epoch 1/100
36/36 [==============================] - ETA: 0s - loss: 0.1635 - acc: 0.9501WARNING:tensorflow:From /usr/local/Anaconda33/lib/python3.7/site-packages/tensorflow/python/ops/resource_variable_ops.py:1817: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /home/gaoboyu/学习保存的模型/77/assets
36/36 [==============================] - 33s 905ms/step - loss: 0.1635 - acc: 0.9501 - val_loss: 0.1574 - val_acc: 0.9505 - lr: 2.0000e-04
Epoch 2/100
36/36 [==============================] - 23s 653ms/step - loss: 0.1625 - acc: 0.9503 - val_loss: 0.1574 - val_acc: 0.9505 - lr: 2.0000e-04
Epoch 3/100
36/36 [==============================] - 24s 653ms/step - loss: 0.1627 - acc: 0.9502 - val_loss: 0.1574 - val_acc: 0.9505 - lr: 2.0000e-04
Epoch 4/100
36/36 [================

KeyboardInterrupt: 

In [29]:
import pickle

history_path = '/home/gaoboyu/学习保存的模型/766/history.txt'
with open(history_path, 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [24]:
cs = history.history.get('loss')

In [25]:
len(cs)

140

In [12]:
# 将数据应用于函数：
AUTOTUNE = tf.data.experimental.AUTOTUNE   # 多线程加载：图像加载是cpu处理的，cpu可以做多线程！
dataset = dataset.map( load_val_image, num_parallel_calls = AUTOTUNE )

In [15]:
BATCH_SIZE = 10
dataset = dataset.cache().batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 256, 256, 3), (None, 256, 256, 1)), types: (tf.float32, tf.int32)>

In [18]:
# model = tf.keras.models.load_model( '/home/gaoboyu/学习保存的模型dataset漠11_8x8_2' )
model = tf.keras.models.load_model( 'model12_450_调过拟合.h5' )

In [19]:
model.evaluate(val_dataset)

9/9 [==============================] - 4s 399ms/step - loss: 0.1807 - acc: 0.9498


[0.18072709441184998, 0.9497509598731995]

In [26]:
# 降速训练1（0.0001）：再次编译 + 再次训练
model.compile(
    optimizer = tf.keras.optimizers.Adam( lr = 0.001 ),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

# 再次训练：
initial_epochs = EPOCHES
final_epochs = 30  # 再多训练10个epoch
total_epochs = initial_epochs + final_epochs

history1 = model.fit(
    train_dataset,
    epochs = total_epochs,
    initial_epoch = initial_epochs,  # 数值从initial_epochs开始记录而已！
    steps_per_epoch = train_count // BATCH_SIZE,
    validation_data = val_dataset,
    validation_steps = val_count // BATCH_SIZE,
    # 回调函数：
    callbacks = [cp_callback]
)

Epoch 141/170
36/36 [==============================] - 59s 2s/step - loss: 0.0789 - acc: 0.9655 - val_loss: 0.1807 - val_acc: 0.9445
Epoch 142/170
36/36 [==============================] - 59s 2s/step - loss: 0.0770 - acc: 0.9664 - val_loss: 0.1745 - val_acc: 0.9451
Epoch 143/170
36/36 [==============================] - 58s 2s/step - loss: 0.0757 - acc: 0.9669 - val_loss: 0.1606 - val_acc: 0.9469
Epoch 144/170
36/36 [==============================] - 58s 2s/step - loss: 0.0762 - acc: 0.9667 - val_loss: 0.1723 - val_acc: 0.9460
Epoch 145/170
36/36 [==============================] - 58s 2s/step - loss: 0.0809 - acc: 0.9645 - val_loss: 0.1734 - val_acc: 0.9439
Epoch 146/170
36/36 [==============================] - 58s 2s/step - loss: 0.0768 - acc: 0.9665 - val_loss: 0.1633 - val_acc: 0.9441
Epoch 147/170
36/36 [==============================] - 58s 2s/step - loss: 0.0768 - acc: 0.9665 - val_loss: 0.1730 - val_acc: 0.9451
Epoch 148/170
36/36 [==============================] - 58s 2s/step - 

In [27]:
import pickle

history_path = '/home/gaoboyu/学习保存的模型/75/history1.txt'
with open(history_path, 'wb') as file_pi:
    pickle.dump(history1.history, file_pi)

In [29]:
cs = history1.history.get('loss')
len(cs)

30

In [31]:
# 降速训练1（0.001）：再次编译 + 再次训练
model.compile(
    optimizer = tf.keras.optimizers.Adam( lr = 0.001 ),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

# 再次训练：
initial_epochs = total_epochs
final_epochs = 30  # 再多训练10个epoch
total_epochs = initial_epochs + final_epochs

history2 = model.fit(
    train_dataset,
    epochs = total_epochs,
    initial_epoch = initial_epochs,  # 数值从initial_epochs开始记录而已！
    steps_per_epoch = train_count // BATCH_SIZE,
    validation_data = val_dataset,
    validation_steps = val_count // BATCH_SIZE,
    # 回调函数：
    callbacks = [cp_callback]
)

Epoch 171/200
36/36 [==============================] - 60s 2s/step - loss: 0.0684 - acc: 0.9702 - val_loss: 0.1630 - val_acc: 0.9465
Epoch 172/200
36/36 [==============================] - 59s 2s/step - loss: 0.0673 - acc: 0.9707 - val_loss: 0.1787 - val_acc: 0.9467
Epoch 173/200
36/36 [==============================] - 58s 2s/step - loss: 0.0670 - acc: 0.9708 - val_loss: 0.1763 - val_acc: 0.9454
Epoch 174/200
36/36 [==============================] - 58s 2s/step - loss: 0.0685 - acc: 0.9702 - val_loss: 0.1715 - val_acc: 0.9459
Epoch 175/200
36/36 [==============================] - 58s 2s/step - loss: 0.0664 - acc: 0.9710 - val_loss: 0.1774 - val_acc: 0.9443
Epoch 176/200
36/36 [==============================] - 58s 2s/step - loss: 0.0680 - acc: 0.9703 - val_loss: 0.1825 - val_acc: 0.9462
Epoch 177/200
36/36 [==============================] - 58s 2s/step - loss: 0.0662 - acc: 0.9711 - val_loss: 0.1762 - val_acc: 0.9478
Epoch 178/200
36/36 [==============================] - 58s 2s/step - 

In [33]:
import pickle

history_path = '/home/gaoboyu/学习保存的模型/75/history2.txt'
with open(history_path, 'wb') as file_pi:
    pickle.dump(history2.history, file_pi)

In [34]:
# 降速训练1（0.001）：再次编译 + 再次训练
model.compile(
    optimizer = tf.keras.optimizers.Adam( lr = 0.001 / 5 ),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

# 再次训练：
initial_epochs = total_epochs
final_epochs = 50  # 再多训练10个epoch
total_epochs = initial_epochs + final_epochs

history3 = model.fit(
    train_dataset,
    epochs = total_epochs,
    initial_epoch = initial_epochs,  # 数值从initial_epochs开始记录而已！
    steps_per_epoch = train_count // BATCH_SIZE,
    validation_data = val_dataset,
    validation_steps = val_count // BATCH_SIZE,
    # 回调函数：
    callbacks = [cp_callback]
)

Epoch 201/250
36/36 [==============================] - 59s 2s/step - loss: 0.0580 - acc: 0.9748 - val_loss: 0.1862 - val_acc: 0.9480
Epoch 202/250
36/36 [==============================] - 59s 2s/step - loss: 0.0568 - acc: 0.9753 - val_loss: 0.1846 - val_acc: 0.9479
Epoch 203/250
36/36 [==============================] - 58s 2s/step - loss: 0.0570 - acc: 0.9752 - val_loss: 0.1886 - val_acc: 0.9477
Epoch 204/250
36/36 [==============================] - 58s 2s/step - loss: 0.0563 - acc: 0.9755 - val_loss: 0.1896 - val_acc: 0.9477
Epoch 205/250
36/36 [==============================] - 58s 2s/step - loss: 0.0560 - acc: 0.9756 - val_loss: 0.1891 - val_acc: 0.9479
Epoch 206/250
36/36 [==============================] - 58s 2s/step - loss: 0.0560 - acc: 0.9756 - val_loss: 0.1866 - val_acc: 0.9480
Epoch 207/250
36/36 [==============================] - 58s 2s/step - loss: 0.0560 - acc: 0.9757 - val_loss: 0.1898 - val_acc: 0.9478
Epoch 208/250
36/36 [==============================] - 58s 2s/step - 

In [35]:
import pickle

history_path = '/home/gaoboyu/学习保存的模型/75/history3.txt'
with open(history_path, 'wb') as file_pi:
    pickle.dump(history3.history, file_pi)

In [36]:
# 降速训练1（0.001）：再次编译 + 再次训练
model.compile(
    optimizer = tf.keras.optimizers.Adam( lr = 0.0001 ),
    loss = 'sparse_categorical_crossentropy',
    metrics = ['acc']
)

# 再次训练：
initial_epochs = total_epochs
final_epochs = 50  # 再多训练10个epoch
total_epochs = initial_epochs + final_epochs

history4 = model.fit(
    train_dataset,
    epochs = total_epochs,
    initial_epoch = initial_epochs,  # 数值从initial_epochs开始记录而已！
    steps_per_epoch = train_count // BATCH_SIZE,
    validation_data = val_dataset,
    validation_steps = val_count // BATCH_SIZE,
    # 回调函数：
    callbacks = [cp_callback]
)

Epoch 251/300
36/36 [==============================] - 60s 2s/step - loss: 0.0510 - acc: 0.9779 - val_loss: 0.2016 - val_acc: 0.9477
Epoch 252/300
36/36 [==============================] - 59s 2s/step - loss: 0.0508 - acc: 0.9779 - val_loss: 0.2044 - val_acc: 0.9476
Epoch 253/300
36/36 [==============================] - 58s 2s/step - loss: 0.0506 - acc: 0.9781 - val_loss: 0.2043 - val_acc: 0.9477
Epoch 254/300
36/36 [==============================] - 58s 2s/step - loss: 0.0508 - acc: 0.9779 - val_loss: 0.2025 - val_acc: 0.9476
Epoch 255/300
36/36 [==============================] - 58s 2s/step - loss: 0.0503 - acc: 0.9781 - val_loss: 0.2058 - val_acc: 0.9474
Epoch 256/300
36/36 [==============================] - 58s 2s/step - loss: 0.0504 - acc: 0.9781 - val_loss: 0.2040 - val_acc: 0.9474
Epoch 257/300
36/36 [==============================] - 58s 2s/step - loss: 0.0503 - acc: 0.9782 - val_loss: 0.2042 - val_acc: 0.9476
Epoch 258/300
36/36 [==============================] - 58s 2s/step - 

In [38]:
history_path = '/home/gaoboyu/学习保存的模型/75/history4.txt'
with open(history_path, 'wb') as file_pi:
    pickle.dump(history4.history, file_pi)